In [9]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/thesis/graph_neural_network/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/thesis/graph_neural_network


In [ ]:
# !pip install torchmetrics
# !pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu124

import torch
import os
print("PyTorch has version {}".format(torch.__version__))

srs_url = f"https://pytorch-geometric.com/whl/torch-{torch.__version__}.html"
!pip install torch_scatter -f $srs_url
!pip install torch_sparse -f $srs_url
!pip install torch_cluster -f $srs_url
!pip install torch_spline_conv -f $srs_url
!pip install torch_geometric

PyTorch has version 2.4.0+cpu
Looking in links: https://pytorch-geometric.com/whl/torch-2.4.0+cpu.html
Looking in links: https://pytorch-geometric.com/whl/torch-2.4.0+cpu.html
     ---------------------------------------- 0.0/801.0 kB ? eta -:--:--
     ---------------------------------------- 801.0/801.0 kB 6.9 MB/s  0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.4.0+cpu.html
Looking in links: https://pytorch-geometric.com/whl/torch-2.4.0+cpu.html
  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
Using cached torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)


In [3]:
!pip uninstall torch_scatter torch_sparse torch_cluster torch_spline_conv torch_geometric -y

Found existing installation: torch-scatter 2.1.2+pt21cpu
Uninstalling torch-scatter-2.1.2+pt21cpu:
  Successfully uninstalled torch-scatter-2.1.2+pt21cpu
Found existing installation: torch-sparse 0.6.18+pt21cpu
Uninstalling torch-sparse-0.6.18+pt21cpu:
  Successfully uninstalled torch-sparse-0.6.18+pt21cpu
Found existing installation: torch-cluster 1.6.3+pt21cpu
Uninstalling torch-cluster-1.6.3+pt21cpu:
  Successfully uninstalled torch-cluster-1.6.3+pt21cpu
Found existing installation: torch-spline-conv 1.2.2+pt21cpu
Uninstalling torch-spline-conv-1.2.2+pt21cpu:
  Successfully uninstalled torch-spline-conv-1.2.2+pt21cpu
Found existing installation: torch-geometric 2.6.1
Uninstalling torch-geometric-2.6.1:
  Successfully uninstalled torch-geometric-2.6.1


In [1]:
import logging

import sys
import itertools
import yaml
import subprocess
import os
import logging

# Path to the config file and train script
config_path = "config.yaml"
train_script = "train.py"


for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.DEBUG,
    datefmt='%m-%d %H:%M:%S',
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s"
)
logger = logging.getLogger(__name__)


In [13]:
### Change config

new_config = {
    'epochs': 1,
    "val_size": 0.2,
    "test_size": 0.2,
    "seed": 100,
    'n_layer': 2,
    'learning_rate': 0.001,
    'item_emb_dim': 128,
    'user_emb_dim': 128,
    "negative_sampling_method": "batch_random",
    "recon_loss": "bpr", # "binary", "bpr", "bce"
    'feature_linear_dim': 128,
    'hidden_channels': 128,
    'latent_dim': 64,
    'kl_beta': 0.5,
    "kl_warmup_epoch" : 30,
    'eval_interval': 1,
    'variational': False,
    'save_model': True
}


with open(config_path, "r") as f:
    config = yaml.safe_load(f)

config.update(new_config)

temp_config_path = f"temp_config.yaml"
with open(temp_config_path, "w") as f:
    yaml.dump(config, f)

In [ ]:

process = subprocess.Popen(
    ["python", "-u", train_script, "--config", temp_config_path],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

# Stream output line by line
for line in process.stdout:
    print(line, end="")  # live printing in notebook

process.wait()  # wait for completion


INFO:__main__:Configuration:
INFO:__main__:{'batch_method': 'binary_link_neighbors',
 'batch_norm': False,
 'batch_size': 128,
 'book_features': [{'name': 'textual_desc'},
                   {'aggr_fn': 'mean', 'name': 'textual_reviews'}],
 'books_filename': 'books_filtered.parquet',
 'coreness_k': 5,
 'decoder': 'inner_product_decoder',
 'descriptions_filename': 'descriptions_filtered.parquet',
 'dir': '../data_sample',
 'disjoint_train_ratio': 0.0,
 'dropout': 0.5,
 'edge_type': 'interactions',
 'embeddings_descriptions_filename': 'embeddings_descriptions_sbert_pt.parquet',
 'embeddings_reviews_filename': 'embeddings_reviews_sbert_pt.parquet',
 'encoder': 'sage_encoder',
 'epochs': 1,
 'eval_interval': 1,
 'feature_aggr_method': 'concat',
 'feature_linear_dim': 128,
 'heads': 2,
 'hidden_channels': 128,
 'interactions_filename': 'interactions.parquet',
 'item_emb_dim': 128,
 'item_feature_linear': True,
 'kl_beta': 0.5,
 'kl_warmup_epoch': 30,
 'language_filter': True,
 'latent_dim':

3221226356